In [ ]:
!pip install nlpaug
!pip install transformers

     |████████████████████████████████| 389kB 12.3MB/s 
     |████████████████████████████████| 1.3MB 13.8MB/s 
     |████████████████████████████████| 2.9MB 43.9MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=80f97195d988a1dcfc950add295d345a10cf6bd47a6f5acb0d84803795d69984
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import itertools
from typing import List, Dict, Union

import nlpaug.augmenter.word as naw
import pandas as pd

In [ ]:
ROOT_PATH = "/content/drive/My Drive/Magíster 2020"

In [ ]:
puc_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puc_train_70.json", orient="records")
puc_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puc_test_30.json", orient="records")

In [ ]:
puc_train["first_sdg"].value_counts()

3     430
11     73
9      66
4      65
15     60
2      35
16     30
14     29
7      25
6      24
13     23
12     18
8      16
10     14
17      8
5       4
1       3
Name: first_sdg, dtype: int64

In [ ]:
aurora_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_train_70.json", orient="records")
aurora_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_test_30.json", orient="records")

In [ ]:
def calculate_augmentation(max_count: int, current_count: int) -> int:
    return (max_count - current_count) // current_count

In [ ]:
AURORA_MAX = aurora_train["first_sdg"].value_counts().max()

1185

In [ ]:
def augment_abstract(row: pd.Series, current_value_counts: pd.Series) -> List[Dict[str, Union[str, int]]]:
    aug = naw.ContextualWordEmbsAug(model_path="distilbert-base-cased", action="substitute")
    sdg = row["first_sdg"]
    title = row["title"]
    current_count = current_value_counts[row["first_sdg"]]
    if current_count == AURORA_MAX:
        return []
    new_samples = calculate_augmentation(AURORA_MAX, current_count)
    augmented_samples = aug.augment(row["clean_abstract"], n=new_samples)
    augmented_samples = [{"title": title, "first_sdg": sdg, "clean_abstract": sample} for sample in augmented_samples]
    return augmented_samples

In [ ]:
puc_value_counts = puc_train["first_sdg"].value_counts()
puc_train["augmented"] = puc_train.apply(lambda row: augment_abstract(row, puc_value_counts), axis=1)

In [ ]:
augmented_flat = list(itertools.chain(*puc_train["augmented"].tolist()))
augmented = pd.DataFrame(augmented_flat)

The cell below correct a bug I don't catch yet, which creates an insane amount of copies of the SDG 3, when it should create just one of each abstract of that SDG.

In [ ]:
augmented_sdg_3 = augmented[augmented["first_sdg"] == 3]
augmented_sdg_3.drop_duplicates("title", keep="first", inplace=True)
augmented_final = augmented[augmented["first_sdg"] != 3]
augmented_final = pd.concat([augmented_final, augmented_sdg_3])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
del puc_train["augmented"]
puc_train = pd.concat([puc_train, augmented_final], ignore_index=True)

In [ ]:
puc_train["first_sdg"].value_counts()

1     1185
17    1184
8     1184
5     1184
6     1176
10    1176
7     1175
13    1173
4     1170
16    1170
12    1170
11    1168
14    1160
2     1155
15    1140
9     1122
3      860
Name: first_sdg, dtype: int64

In [ ]:
puc_train.to_json(f"{ROOT_PATH}/Ordenado/augmented_with_aurora/2020-09-12_puc_augmented.json", orient="records")

In [ ]:
len(puc_train)

19552